# Libs

In [1]:
import numpy as np
import pandas as pd
from PIL import Image
from os.path import join
import random
import zipfile

from torchvision import transforms

In [ ]:
GDRIVE_PATHS = {
    'main': '/content/gdrive/MyDrive/trashbean-classifier/',
    'dataset': '/content/gdrive/MyDrive/trashbean-classifier/dataset/',
    'logs': '/content/gdrive/MyDrive/trashbean-classifier/logs/',
    'models': '/content/gdrive/MyDrive/trashbean-classifier/logs/models/',
    'libs': '/content/gdrive/MyDrive/Colab Notebooks/Progetto/libs'
}

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Insert the directory
import sys
sys.path.insert(0,GDRIVE_PATHS['libs'])

In [6]:
# need do import only concreteCreator
from TrashbeanDataset import TrashbeanDataset
from TDContainer import TDContainer
from PretrainedModels import AverageValueMeter, PretrainedModelsCreator, CCAlexNet, CCSqueezeNet, CCVgg16

In [3]:
random.seed(1996)
np.random.seed(1996)

# Testing

## Load del dataset

In [7]:
train_transform = transforms.Compose([
  transforms.Resize(256),
  transforms.RandomCrop(224),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(224), # crop centrale
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

trashbean_dataset = TDContainer({
  'path': join(GDRIVE_PATHS['dataset'], 'training.csv'),
  'transform': train_transform
},{
  'path': join(GDRIVE_PATHS['dataset'], 'validation.csv'),
  'transform': test_transform
}, {
    'path': join(GDRIVE_PATHS['dataset'], 'test.csv'),
    'transform': test_transform
})

In [8]:
trashbean_dataset.show_info()


=== *** DB INFO *** ===
Training: 900 values, 
Validation: 900 values, 
Test: 900
DataLoader: False
=== *** END *** ====



## Training

In [9]:
def do_training(creator: PretrainedModelsCreator, dataset: TDContainer, output_class: int, dl_attributes: dict, parameters: dict, paths: dict, train_from_epoch: int=0, save_on_runtime: bool=True, save_each_iter:int=20 ) -> None:
    creator.initialize_dst(dataset, output_class, dl_attributes)
    creator.train(parameters=parameters, paths=paths, train_from_epoch=train_from_epoch, save_on_runtime=save_on_runtime, save_each_iter=save_each_iter)

In [21]:
def do_loading(loaded_model, creator: PretrainedModelsCreator, dataset: TDContainer, output_class: int, dl_attributes: dict, parameters: dict, paths: dict, train_from_epoch: int=0, save_on_runtime: bool=True, save_each_iter:int=20 ) -> None:
  creator.initialize_dst(dataset, output_class, dl_attributes)
  creator.load_model(loaded_model)
  creator.train(parameters=parameters, paths=paths, train_from_epoch=train_from_epoch, save_on_runtime=save_on_runtime, save_each_iter=save_each_iter)

In [17]:
# using the Concrete Creator
print("App: Launching training with the SqueezeNet.")
do_training(creator=CCSqueezeNet(), dataset=trashbean_dataset, output_class=3, dl_attributes={'batch_size': 32, 'num_workers': 2, 'drop_last': False}, parameters={'exp_name': 'SqueezeNet__v1', 'lr': 0.001, 'epochs': 1, 'momentum': 0.99 }, paths=GDRIVE_PATHS, train_from_epoch=0, save_on_runtime=True, save_each_iter=1)

App: Launching training with the SqueezeNet.


Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth



Computing epoch:
1 / 1 ... 
Ended in:  1.0570414582888286 minutes
**** Training procedure ended. Start to calculate accuracy ...
Accuracy of SqueezeNet__v1 62.00%


In [22]:
print("App: Reloading training with the SqueezeNet.")
do_loading('/content/gdrive/MyDrive/trashbean-classifier/logs/models/SqueezeNet__v1-1.pth', creator=CCSqueezeNet(), dataset=trashbean_dataset, output_class=3, dl_attributes={'batch_size': 32, 'num_workers': 2, 'drop_last': False}, parameters={'exp_name': 'SqueezeNet__v1', 'lr': 0.001, 'epochs': 1, 'momentum': 0.99 }, paths=GDRIVE_PATHS, train_from_epoch=1, save_on_runtime=True, save_each_iter=1)


App: Reloading training with the SqueezeNet.
Loading model using load_state_dict..
Computing epoch:
1 / 1 ... 
Ended in:  1.0565665721893311 minutes
**** Training procedure ended. Start to calculate accuracy ...
Accuracy of SqueezeNet__v1 33.33%


In [15]:
def make_model_path(path_model, name_model, version):
  return join(path_model, "%s__%s.pth" % (name_model, version))

name_model = 'SqueezeNet'
version="1-0"
print(make_model_path(GDRIVE_PATHS['models'], "SqueezeNet", "v0-999"))